In [ ]:
import gym
import numpy as np
import matplotlib.pyplot as plt
import random

from google.colab import drive

drive.mount("/content/gdrive")

import csv
import time

from os import listdir
from os.path import isfile, join
import numpy as np
from collections import defaultdict

#### Normalize state

In [ ]:
# Returns the state scaled between 0 and 1
def normalize_state(_s):
    _y = np.zeros(len(_s))
    for _i in range(len(_s)):
        _y[_i] = (_s[_i] - xbar[0, _i]) / (xbar[1, _i] - xbar[0, _i])
    return _y

#### Basis Function

In [ ]:
class BasisFunction(object): 
    def __init__(self, discrt, dim, num_actions): 
        
        self.dim = dim
        print("dim", self.dim)
        self.num_actions = num_actions
        print("num_actions", )
        self.num_rbf = discrt * np.ones(self.dim).astype(int)
        self.width = 1. / (self.num_rbf - 1.)

        self.rbf_sigma = self.width[0] / 2.
        
        self.num_ind = np.prod(self.num_rbf)
        self.rbf_den = 2 * self.rbf_sigma ** 2
        
        c = np.zeros((self.num_ind, dim))
        for i in range(self.num_ind):
            if i == 0:
                pad_num = self.dim
            else:
                pad_num = self.dim - int(np.log(i) / np.log(discrt)) - 1
            ind = np.base_repr(i, base=discrt, padding=pad_num)
            ind = np.asarray([float(j) for j in list(ind)])
            c[i, :] = self.width * ind
        
        print("c.shape", c.shape)
        print("c", c)
        
        self.c = c

    def get_activations(self, _state):
        """function phi retrieves a state, and gives feature 
        vector (called activations) as output."""
        _phi = np.zeros(self.num_ind)
        
        for _k in range(self.num_ind):
            _phi[_k] = np.exp(-np.linalg.norm(_state - self.c[_k, :]) ** 2 / 
                              self.rbf_den)
        
        return _phi

#### Greedy policy

In [ ]:
# Returns an action following an epsilon-greedy policy
def epsilon_greedy(_epsilon, _vals):
    _rand = np.random.random()
    if _rand < 1. - _epsilon:
        _action = _vals.argmax()
    else:
        _action = env.action_space.sample()
    return int(_action)

#### Retrieve Q-values

In [ ]:
# Returns the value of each action at some state
def action_values(_activations, _theta):
    _val = np.dot(_theta.T, _activations)
    return _val


# Returns the value of an action at some state
def action_value(_activations, _action, _theta):
    _val = np.dot(_theta[:, _action], _activations)
    return _val

#### Gradient functions

In [ ]:
def semi_gradient(theta, activations, action, reward, done, Q, Q_new, num_ind, gamma, alpha):
    if done:
        target = reward - Q
    else:
        target = reward + Q_new - Q

    for k in range(num_ind):
        theta[k, action] += alpha * target * activations[k]
        
    return theta

def full_gradient(theta, activations, new_activations, action, reward, done, Q, Q_new, num_ind, gamma, alpha):
    if done:
        target = reward - Q
    else:
        target = reward + Q_new - Q

    for k in range(num_ind):
        theta[k, action] -= alpha * target * (gamma * new_activations[k] - activations[k])
        
    return theta


#### Train function

In [ ]:
def train(env, gradient='semi', normalize=True, std=0, alpha=0.01, gamma=1, discrt=4, num_episodes=2000, num_timesteps=400, epsilon=0.1, print_every=10, seed=0):

    filename = f"environment {environment}_{gradient} gradient_std {std}_alpha {alpha}_discrt {discrt}_gamma {gamma} _steps {num_timesteps}_seed {seed}_time {int(time.time())}_epsilon {epsilon}"
    print(filename)

    num_actions = env.action_space.n
    state_dim = env.observation_space.high.size

    bf = BasisFunction(discrt, state_dim, num_actions)
    # rbf_feature = RBFSampler(gamma=1, random_state=1)

    ep_length = []
    ten_eps = []

    theta = np.random.normal(0, std, size=(bf.num_ind, num_actions))
    # theta = np.random.normal(0, std, size=(100, num_actions))
    
    # SARSA loop
    for ep in range(num_episodes):
        
        state = env.reset()
        
        if normalize:
            state = normalize_state(state)
        
        activations = bf.get_activations(state)
        # print("old", activations.shape)
        # activations = rbf_feature.fit_transform(state.reshape(1, -1))[0]
        # print("new", activations.shape)

        eps = epsilon - (epsilon / num_episodes * ep) 

        vals = action_values(activations, theta)
        action = epsilon_greedy(eps, vals)

        # Each episode
        for t in range(num_timesteps):

            new_state, reward, done, info = env.step(action)

            if t == (num_timesteps - 1):
                done = True

            if normalize:
                new_state = normalize_state(new_state)
            
            new_activations = bf.get_activations(new_state)
            # new_activations = rbf_feature.fit_transform(new_state.reshape(1, -1))[0]
        
            new_vals = action_values(new_activations, theta)
            new_action = epsilon_greedy(eps, new_vals)
            
            Q = action_value(activations, action, theta)
            Q_new = action_value(new_activations, new_action, theta)

            if gradient == 'semi':
                theta = semi_gradient(theta, activations, 
                                      action, reward, done, Q, Q_new, 
                                      bf.num_ind, gamma, alpha)
                # print(theta)
            elif gradient == 'full':
                theta = full_gradient(theta, activations, new_activations, 
                                      action, reward, done, Q, Q_new, 
                                      bf.num_ind, gamma, alpha)
            else:   
                raise ValueError('variable "gradient" must be in ["semi", "full"].')

            state = new_state.copy()
            activations = new_activations.copy()
            action = new_action
            
            if done:
                break

        ep_length.append(t)
        ten_eps.append(t)

        if (ep + 1) % print_every == 0: 
            avg_steps = sum(ten_eps) / len(ten_eps)
            ten_eps = []
            print(f"episode {ep}: {avg_steps} steps")
    
    with open(f'/content/gdrive/My Drive/RL/{filename}.csv', 'w') as f:   
        write = csv.writer(f) 
        write.writerow(ep_length)  



#### Plot function

In [ ]:
def plot_results(results):
    for title, result in results.items(): 
        result = np.array(result)
        x = list(range(0, result.shape[1]*gather, gather))

        print(result.shape)
        mu = result.mean(axis=0)
        min_std = mu - result.std(axis=0)
        max_std = mu + result.std(axis=0)

        plt.figure(figsize=(6, 4))
        plt.plot(x, mu)
        plt.fill_between(x, max_std, min_std, alpha=0.5)
        plt.xlabel("episodes")
        plt.ylabel("timesteps")
        print(title)
        plt.title(title)
        plt.show()


#### Train the model

In [ ]:
environment = "MountainCar-v0"
# environment = "CartPole-v1"

env = gym.make(environment).env
# env = gym.make(environment)

# nescesary to normalize the state 
num_actions = env.action_space.n
state_dim = env.observation_space.high.size
print(num_actions, state_dim)

xbar = np.zeros((2, state_dim))
xbar[0, :] = env.observation_space.low
xbar[1, :] = env.observation_space.high

In [ ]:
std=0   
alpha=0.005
discrt=4
gamma=1
num_episodes=1000
num_timesteps=400
epsilon=0.1
print_every=100

for gradient in ['semi', 'full']: 
    for std in [.1]:
        for seed in range(10): 
            env.seed(seed)
            np.random.seed(seed)
            random.seed(seed)
            env.action_space.seed(seed)

            train(env, gradient=gradient, std=std, alpha=alpha, gamma=gamma, discrt=discrt, 
                num_episodes=num_episodes, num_timesteps=num_timesteps, epsilon=epsilon, 
                print_every=print_every, seed=seed)

#### Plot the results 

In [ ]:
mypath = '/content/gdrive/My Drive/RL/'
gather = 10
substrings = ["MountainCar-v0"]

results = defaultdict(list)
files = [f for f in listdir(mypath) if isfile(join(mypath, f)) and all(substr in f for substr in substrings)]

for file in files:
    envrnmnt, gradient, std, alpha, discrt, gamma, steps, seed, timestamp, epsilon = file.split("_")
    
    with open(f'{mypath}{file}') as f:
        reader = csv.reader(f)
        data = list(map(int, list(reader)[0]))
    
    data = list(np.mean(np.array(data).reshape(-1, gather), axis=1))

    results[f"{gradient} with {std}"].append(data)
        
print(list(results.keys()))

In [ ]:
plot_results(results)

#### Get mean episode length

In [ ]:
for title, result in results.items(): 
    print(title)
    result = np.array(result)
    
    print(result.shape)
    mu = result.mean(axis=0)[200:]
    print(mu.mean())
